# Adaptive PDE discretizations on cartesian grids
## Volume : Algorithmic tools
## Part : Tensor decomposition techniques
## Chapter : Voronoi's reduction, in dimension 6

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; TocTools.displayTOC('TensorVoronoi','Algo')

In [2]:
from agd import LinearParallel as lp
from agd.Selling import GatherByOffset
from agd.Plotting import savefig; #savefig.dirName = 'Figures/TensorVoronoi'

The routines for tensor decomposition are for efficiency purposes provided in a small c++ library, named FileVDQ where VDQ stands for "Voronoi Decomposition of Quadratic forms". This is in contrast with the two and three dimensional cases, where the decomposition algorithm is coded in Python (the c++ library can also be used in smaller dimensions). A function named `VoronoiDecomposition` provides the interface.

In [3]:
from agd.Eikonal import VoronoiDecomposition

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 0.1 Optional configuration

Uncomment the following line to use the GPU implementation of Voronoi's decomposition.

In [5]:
VoronoiDecomposition.default_mode = 'gpu_transfer'

## 1. Computing the decomposition of a tensor

We illustrate our tensor decomposition method on random positive definite matrices, of the form 
$$
    D = A^T A,
$$
where $A$ is a square matrix with random coefficients w.r.t. the Gaussian normal law.

In [6]:
def MakeRandomTensor(dim,shape = tuple()):
    A = np.random.standard_normal( (dim,dim) + shape )
    return lp.dot_AA(lp.transpose(A),A)

In [7]:
# For reproducibility, we fix the random seed
np.random.seed(42) 

The inserse operation to tensor decomposition is, of course, reconstruction, defined by 
$$
    (\lambda_i, e_i)_{i=1}^I \mapsto D = \sum_{1 \leq i \leq I} \lambda_i e_i e_i^T
$$

In [8]:
def Reconstruct(coefs,offsets):
     return lp.mult(coefs,lp.outer_self(offsets)).sum(2)

In [9]:
def LInfNorm(a):
    return np.max(np.abs(a))

### 1.1 Case of a  $4 \times 4$ tensor

In [10]:
D4 = MakeRandomTensor(4)

In [11]:
coefs,offsets = VoronoiDecomposition(D4)

Our decomposition, of a $4 \times 4$ SPD tensor, involves either $10$ or $12$ coefficients and offsets. 
If the tensor is randomly generated, then each possibility arises with positive probability, in approximately half the cases.

For uniformity of the data structures, we always return $12$ coefficients and offsets, but the last two are often zero.

In [12]:
print("Coefficients : ", coefs)
print("Offsets : \n", offsets.astype(int))

Coefficients :  [0.00623791 1.739663   0.30215493 0.27498621 0.18107025 0.0664281
 0.00623789 1.3873893  0.93488663 0.16328326 0.0062379  0.15724699]
Offsets : 
 [[ 1  0  0  0  1  1  0  0  0  1  1  1]
 [ 0  1  1  0 -1 -1  1  0  1 -1 -1 -2]
 [ 1  0  1 -1  0  1  1 -1  2 -1  0 -1]
 [ 2  0  1  0  1  2  0 -1  1  1  2  1]]


By design, the coefficients are non-negative, and the reconstruction is exact up to numerical precision.

In [13]:
print("Minimal coefficient : ", np.min(coefs))
print("Reconstruction error : ", LInfNorm(D4-Reconstruct(coefs,offsets)))
assert np.allclose(D4,Reconstruct(coefs,offsets))

Minimal coefficient :  0.006237894296646118
Reconstruction error :  5.549733129717183e-07


### 1.1 Case of a  $6 \times 6$ tensor

In [17]:
nsupport = [21, 30, 36, 21, 27, 22, 21]

In [15]:
#coefs,offsets = VoronoiDecomposition(D6)

In [16]:
assert False
for i in range(300):
    np.random.seed(i)
    D = MakeRandomTensor(6)
    _,_,vertex,_ = VoronoiDecomposition(D,single_step=False)
    if vertex!=2:
        print(i,vertex)

AssertionError: 

286,(268,97,154),255

In [18]:
np.random.seed(255)
D = MakeRandomTensor(6)
m,a,vertex,objective = VoronoiDecomposition(D,steps="Single")
assert vertex!=2
nsupport[vertex]

27

In [19]:
VoronoiDecomposition(D,steps="Split",traits = {"nsupport_max":27})

CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

In [17]:
#coefs,offsets =  VoronoiDecomposition(D,traits = {"nsupport_max":max(23,nsupport[vertex])})
coefs,offsets =  VoronoiDecomposition(D,traits = {"nsupport_max":31})
print("Reconstruction error : ", LInfNorm(D-Reconstruct(coefs,offsets)))

CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

In [ ]:
D

In [ ]:
coefs

In [ ]:
(39*17*14//2)

In [ ]:
#coefs,offsets = VoronoiDecomposition(D6)

Our decomposition, of a $4 \times 4$ SPD tensor, involves either $10$ or $12$ coefficients and offsets. 
If the tensor is randomly generated, then each possibility arises with positive probability, in approximately half the cases.

For uniformity of the data structures, we always return $12$ coefficients and offsets, but the last two are often zero.

In [ ]:
print("Coefficients : ", coefs)
print("Offsets : \n", offsets.astype(int))

By design, the coefficients are non-negative, and the reconstruction is exact up to numerical precision.

In [ ]:
print("Minimal coefficient : ", np.min(coefs))
print("Reconstruction error : ", LInfNorm(D6-Reconstruct(coefs,offsets)))
assert np.allclose(D6,Reconstruct(coefs,offsets))

2 1
21 1
23 1
39 1
40 1
43 1
75 1
82 1
83 1
97 5
99 1
114 1
122 1
131 1
135 1
145 1
154 5
162 1
184 5
186 1
198 3
201 1
205 1
217 1
218 1
235 1
237 1
255 4
261 1
268 5
273 1
279 1
284 1
286 3
298 1

## 1.2 A family of tensors

In [ ]:
def Interpolate(a,b,T=np.linspace(0,1,100)):
    return T, np.moveaxis(np.array([(1-t)*a + t*b for t in T]),0,-1)

In [ ]:
T_interp, D_interp = Interpolate(MakeRandomTensor(6),MakeRandomTensor(6))

In [ ]:
%%time
coefs,offsets = VoronoiDecomposition(D_interp)

In [ ]:
print("Reconstruction error : ", LInfNorm(D_interp - Reconstruct(coefs,offsets)))
assert np.allclose(D_interp, Reconstruct(coefs,offsets),atol=1e-5)

In [ ]:
decomp = GatherByOffset(T_interp,coefs,offsets)

In [ ]:
fig = plt.figure(figsize=(20,10))
for offset,(time,coef) in decomp.items():
    plt.plot(time,coef)
plt.legend(decomp.keys(),ncol=3)
savefig(fig,"Coefs_Vor4.pdf")

In [ ]:
fig = plt.figure(figsize=(20,10))
for offset,(time,coef) in decomp.items():
    plt.plot(time,coef)
plt.legend(decomp.keys(),ncol=3)
savefig(fig,"Coefs_Vor4.pdf")